In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv

In [2]:
#Scrape years from 2017-2018 to 2022- 2023

base_url = "http://www.dc.state.fl.us/pub/mortality/"

hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

def get_urls_from_single_page(date_url):
    page = requests.get(date_url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    url_list = []
    
    year_list = soup.find_all('th', scope ='row')
    for year in year_list:
        url_list.append(year.find('a').attrs['href'])
        
    return url_list

url_list = get_urls_from_single_page(base_url)

print(url_list)
        
        

['2017-2018.html', '2018-2019.html', '2019-2020.html', '2020-2021.html', '2021-2022.html', '2022-2023.html']


In [4]:
#Scrape name, dcNumber, dateOfDeath, institution, mannerOfDeath

list_url = "http://www.dc.state.fl.us/pub/mortality/2022-2023.html"

def scrape_one_incident(incident_url):
    response = requests.get(incident_url, headers=hdr)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    rows = soup.find("table", {"class": "small"}).find_all("tr")
    data = []
    for row in rows[1:]:
        cells = row.find_all("td")
        name = cells[0].text.strip()
        dc_number = cells[1].text.strip()
        date_of_death = cells[2].text.strip()
        institution_name = cells[3].text.strip()
        manner_of_death = cells[4].text.strip()
        data.append({'name': str(name), 'dc_number': str(dc_number), 'date_of_death': str(date_of_death), 'institution_name':str(institution_name), 'manner_of_death':str(manner_of_death)})
    return data

#test url
#url_list = [list_url]

incident_data = []
for url in url_list:
    data = scrape_one_incident("http://www.dc.state.fl.us/pub/mortality/" + url)
    incident_data.extend(data)
    time.sleep(1)

# Print the list of all incident data
print(incident_data)

[{'name': 'CASWELL, HAROLD', 'dc_number': 'C05596', 'date_of_death': '07/01/2017', 'institution_name': 'R.M.C.- MAIN UNIT', 'manner_of_death': 'Natural'}, {'name': 'DAY, MICHAEL', 'dc_number': 'J56153', 'date_of_death': '07/02/2017', 'institution_name': 'SANTA ROSA C.I.', 'manner_of_death': 'Suicide'}, {'name': 'FLOWERS, BERNARD', 'dc_number': '034392', 'date_of_death': '07/03/2017', 'institution_name': 'S.F.R.C SOUTH UNIT', 'manner_of_death': 'Accident'}, {'name': 'CARTER, STANLEY', 'dc_number': '024167', 'date_of_death': '07/05/2017', 'institution_name': 'MADISON C.I.', 'manner_of_death': 'Natural'}, {'name': 'MORALES, RAMON', 'dc_number': '168554', 'date_of_death': '07/05/2017', 'institution_name': 'CALHOUN WC', 'manner_of_death': 'Accident'}, {'name': 'THOMAS, ROBERT', 'dc_number': '001092', 'date_of_death': '07/06/2017', 'institution_name': 'R.M.C.- MAIN UNIT', 'manner_of_death': 'Natural'}, {'name': 'DAVIS, STANLEY', 'dc_number': '431525', 'date_of_death': '07/06/2017', 'institut

In [5]:
# ALL year urls

def get_urls_from_single_page(year_url):
    page = requests.get(year_url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    url_list = []
    
    inmate_list = soup.find_all('table', class_ ='small')
    for inmate in inmate_list:
        atags = inmate.find_all('a')
        link_list = []
        for a in atags: 
            link_list.append(a.attrs['href'])
        url_list.extend(link_list)
        
    return url_list

inmateYearUrls_list = []
for url in url_list:
    data = get_urls_from_single_page("http://www.dc.state.fl.us/pub/mortality/" + url)
    inmateYearUrls_list.extend(data)
    


print(inmateYearUrls_list)

['/OffenderSearch/detail.aspx?Page=Detail&DCNumber=C05596&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=J56153&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=034392&TypeSearch=IR', 'PDF/034392.pdf', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=024167&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=168554&TypeSearch=IR', 'PDF/168554.pdf', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=001092&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=431525&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=578085&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=W19113&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=U37780&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=116761&TypeSearch=IR', 'PDF/116761.pdf', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=021976&TypeSearch=IR', '/OffenderSearch/detail.aspx?Page=Detail&DCNumber=548445&TypeSearch=I

In [19]:
#Scrape name, race, sex, birth date, custody

#test_url = "http://www.dc.state.fl.us/OffenderSearch/detail.aspx?Page=Detail&DCNumber=Y53732&TypeSearch=IR"

def scrape_one_inmate(inmate_url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(inmate_url, headers=hdr)
    soup = BeautifulSoup(response.content, 'html.parser')

    name = ""
    name_tags = soup.find_all('th', scope='row')
    for tag in name_tags:
        if "Name" in tag.text:
            name = tag.find_next_sibling('td').text.strip()
            break
    # add a placeholder string if name is empty
    if not name:
        name = "Unknown"

    race = ""
    race_tags = soup.find_all('th', scope='row')
    for tag in race_tags:
        if "Race" in tag.text:
            race = tag.find_next_sibling('td').text.strip()
            break
    # add a placeholder string if race is empty
    if not race:
        race = "Unknown"

    sex = ""
    sex_tags = soup.find_all('th', scope='row')
    for tag in sex_tags:
        if "Sex" in tag.text:
            sex = tag.find_next_sibling('td').text.strip()
            break
    # add a placeholder string if sex is empty
    if not sex:
        sex = "Unknown"

    bday = ""
    bday_tags = soup.find_all('th', scope='row')
    for tag in bday_tags:
        if "Birth Date" in tag.text:
            bday = tag.find_next_sibling('td').text.strip()
            break
    # add a placeholder string if bday is empty
    if not bday:
        bday = "Unknown"

    custody = ""
    custody_tags = soup.find_all('th', scope='row')
    for tag in custody_tags:
        if "Custody" in tag.text:
            custody = tag.find_next_sibling('td').text.strip()
            break
    # add a placeholder string if custody is empty
    if not custody:
        custody = "Unknown"

    data = {'name': str(name), 'race': str(race), 'sex': str(sex), 'bday':str(bday), 'custody':str(custody)}

    return data

inmateBio_list = []
for url in inmateYearUrls_list:
    data = scrape_one_inmate("http://www.dc.state.fl.us/" + url)
    inmateBio_list.append(data) 

print(inmateBio_list)


[{'name': 'CASWELL, HAROLD R', 'race': 'WHITE', 'sex': 'MALE', 'bday': '08/18/1971', 'custody': 'MEDIUM'}, {'name': 'DAY, MICHAEL J', 'race': 'WHITE', 'sex': 'MALE', 'bday': '09/29/1968', 'custody': 'CLOSE'}, {'name': 'FLOWERS, BERNARD', 'race': 'BLACK', 'sex': 'MALE', 'bday': '08/16/1954', 'custody': 'CLOSE'}, {'name': 'Unknown', 'race': 'Unknown', 'sex': 'Unknown', 'bday': 'Unknown', 'custody': 'Unknown'}, {'name': 'CARTER, STANLEY R', 'race': 'WHITE', 'sex': 'MALE', 'bday': '01/13/1943', 'custody': 'CLOSE'}, {'name': 'MORALES, RAMON', 'race': 'HISPANIC', 'sex': 'MALE', 'bday': '11/28/1970', 'custody': 'MEDIUM'}, {'name': 'Unknown', 'race': 'Unknown', 'sex': 'Unknown', 'bday': 'Unknown', 'custody': 'Unknown'}, {'name': 'THOMAS, ROBERT', 'race': 'BLACK', 'sex': 'MALE', 'bday': '06/07/1945', 'custody': 'CLOSE'}, {'name': 'DAVIS, STANLEY L', 'race': 'BLACK', 'sex': 'MALE', 'bday': '01/30/1965', 'custody': 'CLOSE'}, {'name': 'CAMPBELL, BILLY R', 'race': 'WHITE', 'sex': 'MALE', 'bday': '0

In [11]:
def write_csv():
    filename = "fl_inmates_mortality_info.csv" 
    f = open(filename, 'w', newline='', encoding='utf-8')
    writer = csv.writer(f)
    fieldnames = ['name', 'dc_number', 'date_of_death', 'institution_name', 'manner_of_death']
    writer.writerow(fieldnames)

    all_inmate_data = []
    for incident in incident_data:
        writer.writerow([incident['name'], incident['dc_number'], incident['date_of_death'], incident['institution_name'], incident['manner_of_death']])


    f.close()

write_csv()


In [22]:
def write_csv():
    filename = "fl_inmates_mortality_bio.csv" 
    f = open(filename, 'w', newline='', encoding='utf-8')
    writer = csv.writer(f)
    fieldnames = ['name', 'race', 'sex', 'bday', 'custody']
    writer.writerow(fieldnames)

    for inmate in inmateBio_list:
        writer.writerow([inmate['name'], inmate['race'], inmate['sex'], inmate['bday'], inmate['custody']])

    f.close()

write_csv()

